In [ ]:
import os
path = os.path.abspath('')
items = os.listdir(path)
if 'Funcoes_GETS.py' not in items:
    ! jupyter nbconvert --to 'python' Funcoes_GETS.ipynb
import Funcoes_GETS as fg

import pandas as pd
import numpy as np
import ipywidgets as widgets
from bqplot import pyplot as bqplt
import bqplot as bq
%matplotlib notebook  

In [ ]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', UserWarning)

In [ ]:
'''###########################
   ####Update data to plot####
   ###########################'''
def update_data(equip,start_date,end_date):
    import pandas as pd
    import numpy as np
    from datetime import date
    today = pd.to_datetime(date.today()).to_numpy()
    available_equips, equip_amount_data, break_data = fg.get_available_equip(equip,equips_data,OS_data)

    #arrange equip whole data
    x_data1 = equip_amount_data.index.get_level_values(0).values
    y_data1 = equip_amount_data['Quantidade de Equipamentos'].values.astype(int)

    
    x_data2 = available_equips.index.values
    y_data2 = available_equips['Quantidade Disponível'].values.astype(int)

    #arrange break_data the same way
    #create time series for break_data
    if break_data.empty==False:
        break_ts = pd.Series(break_data['Taxa de Quebra'].values, index=break_data.index.get_level_values(0))
        #insert last datapoint as today, replicating last known value
    else:
        break_ts = pd.DataFrame()
    return(x_data1,y_data1,x_data2,y_data2, break_ts)

def update_data2(equip,start_date,end_date):
    import pandas as pd
    from datetime import date
    today = pd.to_datetime(date.today()).to_numpy()
    available_equips, equip_amount_data, break_data = fg.get_available_equip(equip,equips_data,OS_data)
    lacking_inflation_corr = False
    material_monthly_cost, lacking_inflation_corr = fg.get_equip_monthly_cost(equip,material_cost_data,ipca)
    external_monthly_cost, lacking_inflation_corr = fg.get_equip_monthly_cost(equip,external_cost_data,ipca)
    if lacking_inflation_corr==True:
        if ipca.empty:
            load_msg = 'Atenção! Inflação não corrigida! '
            load_label.value = f"<b><font color='orange'>{load_msg}</b>" #r'\(\color{orange} {' + load_msg + '}\)'
        else:
            last_corrected_date = ipca.index[-1].strftime('%B %Y')
            last_corrected_month = month_name_dict[ipca.index[-1].month_name()]
            last_corrected_year = str(ipca.index[-1].year)
            load_msg = 'Atenção! Inflação corrigida até ' + last_corrected_month + last_corrected_year
            load_label.value = f"<b><font color='orange'>{load_msg}</b>"#r'\(\color{orange} {' + load_msg + '}\)'
    x_data1,y_data1,x_data2,y_data2,x_data3,y_data3 = fg.arrange_cost_data_to_plot(material_monthly_cost,external_monthly_cost,equip_amount_data)

    #Ensures that material cost dates and ext service cost dates start together (insert zeros in one of them if the other starts earlier)
    if (len(x_data2)!=0) & (len(x_data3)!=0):
        if x_data2[0]!=x_data3[0]:
            #Check if material data starts before ext_services
            mat_starts_before = np.argmin(np.array([x_data3[0],x_data2[0]])).astype(bool)
            if mat_starts_before:
                #If it does, insert earlier dates into ext_services dates and fills ext_services values with zeros
                for d in pd.date_range(x_data2[0],x_data3[0],freq='M')[:-1][::-1]:
                    x_data3 = np.insert(x_data3,0,d)
                    y_data3 = np.insert(y_data3,0,0)
            else:
                #Otherwise, insert earlier dates into material dates and fills material values with zeros
                for d in pd.date_range(x_data3[0],x_data2[0],freq='M')[:-1][::-1]:
                    x_data2 = np.insert(x_data2,0,d)
                    y_data2 = np.insert(y_data2,0,0)
            #Check if material data ends after ext_services
            mat_ends_after = np.argmax(np.array([x_data3[-1],x_data2[-1]])).astype(bool)
            if mat_ends_after:
                for d in pd.date_range(x_data3[-1],x_data2[-1],freq='M')[1:]:
                    x_data3 = np.append(x_data3,d.to_numpy())
                    y_data3 = np.append(y_data3,0)
            else:
                for d in pd.date_range(x_data2[-1],x_data3[-1],freq='M')[1:]:
                    x_data2 = np.append(x_data2,d.to_numpy())
                    y_data2 = np.append(y_data2,0)
                x_data3 = x_data3.astype(np.datetime64)
    elif (len(x_data2)!=0) & (len(x_data3)==0):
        x_data3 = np.copy(x_data2)
        y_data3 = np.zeros_like(y_data2)
    elif (len(x_data2)==0) & (len(x_data3)!=0):
        x_data2 = np.copy(x_data3)
        y_data2 = np.zeros_like(y_data3)
    #arrange break_data the same way
    #create time series for break_data
    break_ts = pd.Series(break_data['Taxa de Quebra'].values, index=break_data.index.get_level_values(0))
    return(x_data1,y_data1,x_data2,y_data2,x_data3,y_data3,break_ts)
def update_data3(equip):
    import pandas as pd
    import numpy as np
    from datetime import date
    today = pd.to_datetime(date.today()).to_numpy()
    available_equips, equip_amount_data, break_data = fg.get_available_equip(equip,equips_data,OS_data)
    duration_data = fg.get_equip_OS_duration(equip,OS_duration_data)
    #arrange equip whole data
    x_data1 = equip_amount_data.index.get_level_values(0).values
    y_data1 = equip_amount_data['Quantidade de Equipamentos'].values.astype(int)

    x_data2 = duration_data.index
    y_data2 = duration_data['Tempo SOS-OSP (horas)'].values

    #arrange break_data the same way
    #create time series for duration_data
    if duration_data.empty==False:
        duration_ts = pd.Series(duration_data['Tempo SOS-OSP (horas)'].values, index=duration_data.index)
    else:
        duration_ts = pd.DataFrame()
    return(x_data1,y_data1,x_data2,y_data2, duration_ts)


'''###########################
   #######Create widgets######
   ###########################'''
title = widgets.Label(value="Interface de Análise Temporal do GETS",
                      layout=widgets.Layout(border= '2px solid black',width='100%', display='flex', align_items='center',
                                           justify_content='center'))
tab_titles = ['Quantidade','Custo','Duração']
tab = widgets.Tab()
[tab.set_title(i, title) for i, title in enumerate(tab_titles)]
fig = bq.Figure(animation_duration=1000,axes=[])
x_dt = bq.DateScale()
y_sc1 = bq.LinearScale()
def_tt = bq.Tooltip(fields=['x', 'y'], formats=['%d-%b-%y', 'd'],labels=['Data','Quant.'])
scatter1 = bqplt.scatter(x=np.array([0,1]),y=np.array([0,1]),colors=['#ff77ff00'], stroke_width=1, display_legend=False,
                                      scales={'x': x_dt, 'y': y_sc1}, tooltip=def_tt)
scatter1_edges = bq.Scatter(x=np.array([0,1]),y=np.array([0,1]),colors=['#ff77ff00'], stroke='steelblue', stroke_width=1, display_legend=False,
                                      scales={'x': x_dt, 'y': y_sc1},marker='rectangle',default_size=20)
scatter2 = bq.Scatter(x=np.array([0,1]),y=np.array([0,1]),colors=['#ff77ff00'], stroke_width=1, display_legend=False,
                                  scales={'x': x_dt, 'y': y_sc1}, tooltip=def_tt)
scatter2_edges = bq.Scatter(x=np.array([0,1]),y=np.array([0,1]),colors=['#ff77ff00'], stroke='coral', stroke_width=1, display_legend=False,
                                      scales={'x': x_dt, 'y': y_sc1},marker='rectangle',default_size=20)
interp = 'linear'
line1 = bq.Lines(x=np.array([0,1]),y=np.array([0,1]),scales={'x': x_dt, 'y': y_sc1}, display_legend=True, labels=['Ativo'],
                             interpolation=interp, colors=['steelblue'])
line2 = bq.Lines(x=np.array([0,1]),y=np.array([0,1]),scales={'x': x_dt, 'y': y_sc1}, display_legend=True, 
                             labels=['Disponivel'],interpolation=interp, colors=['coral'])
ax_x = bq.Axis(scale=x_dt, label='Data', 
                          tick_style={'font-size': 14}, tick_format="%b %Y", tick_rotate=-45, label_offset='3.5em')
ax_y1 = bq.Axis(scale=y_sc1, orientation='vertical', label='Quantidade de Equipamentos', 
                           label_style={ 'font-size': 14},
                           tick_style={'font-size': 14},grid_lines='none')
line_forecast = bq.Lines(x=np.array([0,1]),y=np.array([0,1]),scales={'x': x_dt, 'y': y_sc1}, display_legend=True, labels=['Tendencia'],
                             interpolation=interp, colors=['darkred'])
line_forecast_in_sample = bq.Lines(x=np.array([0,1]),y=np.array([0,1]),scales={'x': x_dt, 'y': y_sc1}, display_legend=True, labels=['Tendencia'],
                             interpolation=interp, colors=['darkblue'])
line_conf_int = bq.Lines(x=np.array([0,1]),y=[np.array([0,1]),np.array([0,1])],scales={'x': x_dt, 'y': y_sc1}, 
                         display_legend=False, labels=['IC95%','None'],
                         interpolation=interp, colors=['darkred'], fill='between',fill_opacities=[0.2],stroke_width=0)
scatter_pred_start_point = bq.Scatter(x=np.array([0,1]),y=np.array([0,1]),colors=['#ff77ff00'], stroke='darkred', display_legend=False,
                                  scales={'x': x_dt, 'y': y_sc1}, tooltip=def_tt, marker='circle',default_size=20)

buttonWidth = '60px'
save_data_button = widgets.Button(
    description = 'Salvar Dados',
    tooltip='Salva os dados da figura em documento',
    icon = 'fa-file-excel-o'
)
save_data_button.layout.width = '90px'
save_button = widgets.Button(
    description = 'Salvar',
    button_style='success'
)

save_fig_button= widgets.Button(
    description = 'Salvar Figura',
    tooltip='Salva a figura atual',
    icon = 'file-image-o',
)
save_fig_button.layout.width = '90px'
save_fig_button2= widgets.Button(
    description = 'Salvar Figura',
    tooltip='Salva a figura atual',
    icon = 'file-image-o',
)
save_fig_button2.layout.width = '90px'
tb = widgets.HBox(justify_content='center')
fig2 = bq.Figure(animation_duration=1000,axes=[])
y_sc2 = bq.LinearScale()
def_tt2 = bq.Tooltip(fields=['x', 'y'], formats=['%b %Y', '.2f'],labels=['Mês/Ano','Custo'])
def_tt3 = bq.Tooltip(fields=['x', 'y'], formats=['%Y', '.2f'],labels=['Ano','Custo'])
bar2 = bq.Bars(x=np.array([0,1]),y=np.array([0,1]), scales={'x': x_dt, 'y': y_sc2},tooltip=def_tt2,display_legend=True, 
                             labels=['Materiais'])

bar3 = bq.Bars(x=np.array([0,1]),y=np.array([0,1]), scales={'x': x_dt, 'y': y_sc2},tooltip=def_tt2,display_legend=True, 
                             labels=['Servicos Externos'])
ax_y2 = bq.Axis(scale=y_sc2, orientation='vertical',  
                            side='right', label_color='seagreen',color='seagreen',label_offset='3.5em',
                           tick_style={'stroke': 'seagreen', 'font-size': 12},grid_lines='dashed',grid_color='seagreen')
bar = bq.Bars(x=np.array([0,1]),y=[np.array([0,1]),np.array([0,1])], scales={'x': x_dt, 'y': y_sc2}, type='grouped',
                              colors=['seagreen','yellowgreen'],tooltip=def_tt2,display_legend=True, 
                             labels=['Materiais','Servicos Externos']) 

tb2 = widgets.HBox(justify_content='center')

fig3 = bq.Figure(animation_duration=1000,legend_text={'font-size': 10}, legend_style={'stroke-width': 0},legend_location = 'top-left')
fig3.max_aspect_ratio = 16/9
fig3.layout.object_fit = 'contain'
fig3.layout.height = '40%'
x_dt3 = bq.DateScale()
y_sc3 = bq.LinearScale()
ax_x3 = bq.Axis(scale=x_dt3, label='Ano',tick_format="%Y")
ax_y3 = bq.Axis(scale=y_sc3, orientation='vertical', label='Custos',label_offset='3.8em',grid_lines='none',
                label_style={'font-size': 8, 'color': 'seagreen'}, tick_style={'font-size': 10})

'''########Third tab items######'''
fig4 = bq.Figure(animation_duration=1000,axes=[])
y_sc4 = bq.LinearScale()
def_tt4 = bq.Tooltip(fields=['x', 'y'], formats=['%b %Y', 'd'],labels=['Data','Duracao'])
interp = 'linear'
line4 = bq.Bars(x=np.array([0,1]),y=np.array([0,1]),scales={'x': x_dt, 'y': y_sc4}, tooltip=def_tt4,display_legend=True, 
                             labels=['Duracao'], colors=['purple'])
ax_y4 = bq.Axis(scale=y_sc4, orientation='vertical',  
                side='right', label_color='purple',color='purple',label_offset='3.5em',label='Mediana Mensal da Duracao de OS em horas',
                tick_style={'font-size': 12},grid_lines='none',grid_color='purple')
tb4 = widgets.HBox(justify_content='center')

month_name_dict ={'January':'Janeiro','February':'Fevereiro','March':'Março','April':'Abril','May':'Maio','June':'Junho',
                 'July':'Julho','August':'Agosto','September':'Setembro','October':'Outubro','November':'Novembro',
                 'December':'Dezembro'}

def display_month_graphic(self, target):
    global x_series, x_years, y2, y3
    
    if target['event']=='background_click':
        with fig3.hold_sync():
            fig3.axes = []
            fig3.marks = []
            fig3.title = ''
    else:
        x1,y1,x2,y2,x3,y3,break_ts = update_data2(selected_equips.options,selected_start_date.value,selected_end_date.value)
        # if any data is empty, fill with zeros the other one
        # if both have data, built time array comprising both data, fill with zeros missing y values
        if (len(x2)==0) & (len(x3)!=0):
            x = x3
            y2 = np.zeros_like(y3)
        elif (len(x3)==0) & (len(x2)!=0):
            x = x2
            y3 = np.zeros_like(y2)
        elif (len(x2)!=0) & (len(x3)!=0):
            earliest_date = np.amin(np.concatenate([x2, x3]))
            latest_date = np.amax(np.concatenate([x2, x3]))
            x = pd.date_range(earliest_date,latest_date,freq='M').values
            y22 = np.zeros(len(x))
            y33 = np.zeros(len(x))
            y22[np.isin(x,x2)] = y2
            y33[np.isin(x,x3)] = y3
            y2 = y22
            y3 = y33
        else:
            x = x2
        
        if selected_analysis.value==1:
            selected_bar = bar2
            new_labels = ['Materiais']
            new_colors = ['seagreen']
#             x_series = pd.Series(x2)
        elif selected_analysis.value==2:
            selected_bar = bar3
            new_labels = ['Servicos Externos']
            new_colors = ['yellowgreen']
#             x_series = pd.Series(x3)
        else:
            selected_bar = bar
            new_labels = ['Materiais','Servicos Externos']
            new_colors = ['seagreen','yellowgreen']
            
        selected_month = pd.DatetimeIndex([selected_bar.x[target['data']['index']]]).month
        selected_year = pd.DatetimeIndex([selected_bar.x[target['data']['index']]]).year
        month_name = pd.DatetimeIndex([selected_bar.x[target['data']['index']]]).month_name()[0]
        x_series = pd.Series(x)
        selected_month_indexes = x_series.dt.month[x_series.dt.month==selected_month[0]].index
        x_years = pd.to_datetime(x_series[selected_month_indexes].dt.year,format='%Y')
        # set time interval from first record to end date
        x_dt3.min = pd.to_datetime(x_series.values[0].astype('datetime64[Y]'))
        x_dt3.max = pd.to_datetime(selected_end_date.value.year,format='%Y')
        y2 = y2[selected_month_indexes]
        y3 = y3[selected_month_indexes]
        #if y2 and y3 lack data until selected end date, fill them with zeros to display points on the figure
        #first build dates vector 
        # if the last value in x_years is smaller than the selected end date -> add missing years up to end date
        #   and add zeros to y vectors
        if pd.to_datetime(x_years.values[-1])<pd.to_datetime(selected_end_date.value.year,format='%Y'):
            x_years2 = pd.date_range(pd.to_datetime(x_years.values[-1]), end=pd.to_datetime(selected_end_date.value.year,format='%Y'), freq='YS')[1:].values
            x_years = np.concatenate([x_years,x_years2])
            zeros = np.zeros(len(x_years2))
            y2 = np.concatenate([y2,zeros])
            y3 = np.concatenate([y3,zeros])
            x_years = pd.Series(x_years)
        
        if selected_analysis.value==3:
            line3 = bq.Lines(x=x_years,
                             y=[y2,y3],
                             stroke_width=1, display_legend=True,
                             scales={'x': x_dt3, 'y': y_sc3},
                             labels=new_labels,colors=new_colors)#,marker = 'circle')
            scatter3 = bq.Scatter(x=x_years,
                                  y=y2, 
                                  scales={'x': x_dt3, 'y': y_sc3},colors=[new_colors[0]], tooltip = def_tt3)
            scatter4 = bq.Scatter(x=x_years,
                                  y=y3, 
                                  scales={'x': x_dt3, 'y': y_sc3},colors=[new_colors[1]], tooltip = def_tt3)
            selected_year_idx = x_years.dt.strftime('%Y').tolist().index(str(selected_year[0]))
            scatter4.selected_style={'opacity': 1.0, 'stroke': 'Red'} #'fill': 'DarkOrange',
            scatter4.selected = [selected_year_idx]

            ymax = np.amax(np.concatenate((y2,y3)))
        else:
            if selected_analysis.value==1:
                y = y2
            else:
                y = y3
            line3 = bq.Lines(x=x_years,y=y, 
                            stroke_width=1, display_legend=True,
                            scales={'x': x_dt3, 'y': y_sc3},
                            labels=new_labels,colors=new_colors)#,marker = 'circle')
            scatter3 = bq.Scatter(x=x_years,y=y, 
                            scales={'x': x_dt3, 'y': y_sc3},
                            colors=new_colors, tooltip = def_tt3)
            ymax = np.amax(y)

            selected_year_idx = x_years.dt.strftime('%Y').tolist().index(str(selected_year[0]))
        scatter3.selected_style={'opacity': 1.0, 'stroke': 'Red'}

        scatter3.selected = [selected_year_idx]
            
        y_sc3.max = np.around(1.25*ymax)
        fig3.axes = [ax_x3,ax_y3]
        if selected_analysis.value==3:
            fig3.marks = [line3,scatter3,scatter4]
        else:
            fig3.marks = [line3,scatter3]
    
        fig3.title = month_name_dict[month_name]
    

bar2.on_element_click(display_month_graphic)
bar3.on_element_click(display_month_graphic)
bar.on_element_click(display_month_graphic)
bar2.on_background_click(display_month_graphic)
bar3.on_background_click(display_month_graphic)
bar.on_background_click(display_month_graphic)

file_types = [('.csv',1), ('.xls',2),('.txt',3)]
selected_file_type = widgets.Dropdown(
    options=file_types,
    value=1,
    description='Tipo:',
    disabled=False,
)

add_equips = widgets.Combobox(
    options=[''],
    placeholder='Ex: VENTILADOR UTI',
    value='',
    ensure_option=True,
    disabled=True
)

selected_equips = widgets.SelectMultiple(
    options=[],
    value=[],
    #rows=10,
    description='',
    disabled=True
)

label1 = widgets.Label(value="Tipo de Medida:")
label2 = widgets.Label(value="Equipamento:")
label3 = widgets.Label(value="Data Inicial:")
label4 = widgets.Label(value="Data Final:")
label5 = widgets.Label(value="Mostrar Tendência?")
label6 = widgets.Label(value="Prever a partir de:",layout=widgets.Layout(visibility='hidden'))
label7 = widgets.Label(value="por ",layout=widgets.Layout(flex='1 0 auto', width='auto',visibility='hidden'))
label8 = widgets.Label(value="semanas",layout=widgets.Layout(flex='1 0 auto', width='auto',visibility='hidden'))
label9 = widgets.Label(value='Logs',layout=widgets.Layout(flex='1 0 auto'))
load_msg = ''
load_label = widgets.HTML(value=load_msg)
error_msg = ''
error_label = widgets.HTML(value= '',layout=widgets.Layout(flex='1 0 auto'))
save_msg = ''
save_label = widgets.HTML(value = f"<b><font color='red'>{save_msg}</b>")
# save_label = widgets.Label(value=save_msg)

analysis_types = [('Custo Total de Material/Mes',1), ('Custo Total de Servico Externo/Mes',2),('Custos Totais/Mes',3)]
selected_analysis = widgets.Dropdown(
    options=analysis_types,
    value=3,
    description='',
    disabled=True,
)

tendency = widgets.Checkbox(
    value=False,
    description='Mostrar linha de tendência',
    disabled=True,
    indent=False
)
selected_start_date = widgets.DatePicker(
    description='',
    disabled=True,
)

import pandas as pd
from datetime import date
today = pd.to_datetime(date.today())#.to_numpy()
selected_end_date = widgets.DatePicker(
    description='',
    disabled=True,
    value=today
)
selected_tendency_start_date = widgets.DatePicker(
    description='',
    disabled=False,
    layout=widgets.Layout(visibility='hidden')
)
prediction_range = widgets.BoundedIntText(
    value=52,
    min=1,
    max=104,
    step=1,
    description='',
    disabled=False,
    layout=widgets.Layout(flex='0 1 auto', width='auto',visibility='hidden')
)
predict_button = widgets.Button(
    description = 'Gera Previsão',
    button_style='',
    tooltip='Gera melhor previsão ARIMA ou ETS (isso pode demorar um pouco!)',
    icon='line-chart',
    layout=widgets.Layout(visibility='hidden')
)
prediction_log = widgets.Textarea(
    value='',
    placeholder='Models tested',
    description='',
    disabled=True,
    layout=widgets.Layout(heigth='100%',width='90%')
)
log_box = widgets.HBox([label9,prediction_log],
                       layout=widgets.Layout(justify_content='space-between',
                                             align_items='stretch',
                                             flex='2 0 auto',
                                             visibility='hidden'))

'''###########################
   ####Create load file chooser####
   ###########################'''
# from ipyfilechooser import FileChooser
# # Create and display a FileChooser widget
# fc_load = FileChooser(os.path.join(os.path.abspath(os.getcwd()),'data'))
# # Show or hide folder icons
# fc_load.use_dir_icons = True
# # # Switch to folder-only mode
# fc_load.show_only_dirs = False
# # fc_load.default_filename = add_equips.value + ' output'
# # Change the title (use '' to hide)
# fc_load.title = 'Selecione o arquivo .zip com as tabelas do GETS:'

load_data_button = widgets.Button(
    description = 'Carregar Tabelas',
    tooltip='Carrega as tabelas do GETS a partir do arquivo ".zip"'
)

load_title = widgets.Label(value='Clique no botão "Carregar Tabelas" após ter feito o upload do arquivo ".zip" para a pasta "data"')

load_box = widgets.VBox([load_title,load_data_button,load_label],layout=widgets.Layout(visibility='visible'))

def on_load_data(b):
#     import datetime
    import time, glob, os
    global equips_data, OS_data, material_cost_data, external_cost_data, ipca, OS_duration_data, ipca_flag, data_folder
    ipca_flag = True
    load_msg = 'Carregando dados... Por favor aguarde...'
    load_label.value = f"<b><font color='orange'>{load_msg}</b>" #r'\(\color{orange} {' + load_msg + '}\)'
    external_path_list = []
    equip_path_list = []
    material_path_list = []
    closed_OS_path_list = []
    open_OS_path_list = []
    ipca_path_list = []
    i=0
#     import rarfile as rf
    import zipfile as zf
    zip_path = os.path.join(os.path.abspath(os.getcwd()),'data')
    if not glob.glob(zip_path + '/*.zip'):
        load_msg = 'Erro! Não há arquivo ".zip" na pasta data !'
        load_label.value = f"<b><font color='red'>{load_msg}</b>" #r'\(\color{red} {' + load_msg + '}\)'
        return
#     if fname.endswith('.true'):
    zip_file_path = glob.glob(zip_path + '/*.zip')[0]
    files = zf.ZipFile(zip_file_path, 'r')
    files.extractall(zip_path)
    files.close()
    
    zip_file_name = os.path.splitext(os.path.split(zip_file_path)[-1])[0]
    data_folder = zip_path
    #check if zip files were inside a folder
    new_folder_created = False
    if zip_file_name in os.listdir(data_folder):
        data_folder = os.path.join(data_folder,zip_file_name)
        new_folder_created = True
    for file_name in os.listdir(data_folder):
        if file_name.endswith(".xls") or file_name.endswith(".xlsx"):
            file_path = os.path.join(data_folder, file_name)
            file = pd.read_excel(file_path,skiprows=1,header=None)
            line = file.iloc[0][~pd.isna(file.iloc[0])].values[0]
            if isinstance(line, str):
                if line=='Agendamentos':
                    external_path_list.append(file_path)
                elif line=='Inventário Equipamentos':
                    equip_path_list.append(file_path)
                elif (line.find('Material Utilizado') != -1):
                    material_path_list.append(file_path)
                elif (line.find('Encerrada') != -1):
                    closed_OS_path_list.append(file_path)
                elif (line.find('Pendente') != -1):
                    if len(open_OS_path_list)==1:
                        if os.path.getmtime(file_path)>os.path.getmtime(open_OS_path_list[0]):
                            open_OS_path_list.pop()
                            open_OS_path_list.append(file_path)
                    else:
                        open_OS_path_list.append(file_path)
                elif (line.find('IPCA') != -1):
                    ipca_path_list.append(file_path)
                else:
                    i+=1
                
    
    if len(external_path_list)==0:
        load_msg = 'Erro! Tabela de Conserto Externo faltando!'
        load_label.value = f"<b><font color='red'>{load_msg}</b>" #r'\(\color{red} {' + load_msg + '}\)'
        return
    elif len(material_path_list)==0:
        load_msg = 'Erro! Tabela de Material Utilizado faltando!'
        load_label.value = f"<b><font color='red'>{load_msg}</b>" #r'\(\color{red} {' + load_msg + '}\)'
        return
    elif len(equip_path_list)==0:
        load_msg = 'Erro! Tabela de Lista de Equipamentos faltando!'
        load_label.value = f"<b><font color='red'>{load_msg}</b>" #r'\(\color{red} {' + load_msg + '}\)'
        return
    elif len(closed_OS_path_list)==0:
        load_msg = 'Erro! Tabela de OS Ecerrada faltando!'
        load_label.value = f"<b><font color='red'>{load_msg}</b>" #r'\(\color{red} {' + load_msg + '}\)'
        return
    elif len(open_OS_path_list)==0:
        load_msg = 'Erro! Tabela de OS Pendente faltando!'
        load_label.value = f"<b><font color='red'>{load_msg}</b>" #r'\(\color{red} {' + load_msg + '}\)'
        return
    elif len(ipca_path_list)==0:
        ipca_flag = False
    
    equips_data = fg.get_all_equips_data(equip_path_list)
    OS_data, OS_duration_data = fg.get_all_OS_data(closed_OS_path_list,open_OS_path_list)
    OS_data = OS_data.dropna(subset=['Tipo Equip.'])
    OS_duration_data = OS_duration_data.dropna(subset=['Tipo Equip.'])
    material_cost_data = fg.load_material_cost_data(material_path_list)
    external_cost_data = fg.load_external_cost_data(external_path_list)
    equip_type = np.sort(equips_data['Tipo Equipamento'].unique())
    add_equips.options = equip_type.tolist()
    if ipca_flag==True:
        ipca = fg.load_ipca(ipca_path_list)
    
    # Delete all .xls files in folder data after load
#     path_to_folder = os.path.dirname(fc_load.selected)#os.path.join(os.path.abspath(os.getcwd()),'data')
    ftype = data_folder + '/*.xls*'
    files = glob.glob(ftype, recursive=True)
    #Remove files
    for f in files:
        try:
            os.remove(f)
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))
    #Remove zip file
    try:
        os.remove(zip_file_path)
    except OSError as e:
        print("Error: %s : %s" % (zip_file_path, e.strerror))
    #Remove zip_folder (if it was generated)
    if new_folder_created==True:
        try:
            os.rmdir(data_folder)
        except OSError as e:
            print("Error: %s : %s" % (data_folder, e.strerror))

    load_msg='Pronto!'
    load_label.value = f"<b><font color='green'>{load_msg}</b>" #r'\(\color{green} {' + load_msg + '}\)'
    time.sleep(0.5)

    if ipca_flag==True:
        load_msg=''
    else:
        load_msg = 'Atenção! Tabela IPCA faltando!'
        ipca = pd.Series(dtype=float)
    load_label.value = f"<b><font color='orange'>{load_msg}</b>" #r'\(\color{orange} {' + load_msg + '}\)'

    add_equips.disabled = False
    selected_equips.disabled = False
    selected_analysis.disabled = False
    selected_start_date.disabled = False
    selected_end_date.disabled = False
    tendency.disabled = False
    help_button.disabled = False

load_data_button.on_click(on_load_data)

'''###########################
   ####Create save file chooser####
   ###########################'''
# # from ipyfilechooser import FileChooser
# # Create and display a FileChooser widget
# fc = FileChooser(os.path.join(os.path.abspath(os.getcwd()),'data'))
# # Show or hide folder icons
# fc.use_dir_icons = True
# # # Switch to folder-only mode

# fc.default_filename = add_equips.value + ' output'
# # Change the title (use '' to hide)
# fc.title = 'Salvar como...'




data_type_and_save_box = widgets.HBox([selected_file_type,save_button])
save_box = widgets.VBox([data_type_and_save_box,save_label])

'''###########################
   ######Create toolbars######
   ###########################'''

'''########Toolbar 1########'''
from bqplot.interacts import PanZoom
from traitlets import link
from collections import OrderedDict

pz = PanZoom()
pzx = PanZoom()
pzy = PanZoom()

zoom_interacts = widgets.ToggleButtons(
                                        options=OrderedDict([
                                            ('xy ', pz), 
                                            ('x ', pzx), 
                                            ('y ', pzy),   
                                            (' ', None)]),
                                            icons = ["arrows", "arrows-h", "arrows-v", "stop"],
                                            tooltips = ["zoom/pan em x & y", "zoom/pan em x apenas", "zoom/pan em y apenas", "cancela zoom/pan"]
                                        )
zoom_interacts.index = 3
zoom_interacts.style.button_width = buttonWidth
ResetZoomButton = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Reseta zoom',
    icon='refresh'
)

global ymin, ymax, ymax2
ymin = 0
ymax = 1
ymax2 = 1
def resetZoom(new):
    # Reset the x and y axes on the figure
    fig.axes[0].scale.min = selected_start_date.value
    fig.axes[1].scale.min = float(ymin)
    fig.axes[0].scale.max = selected_end_date.value
    fig.axes[1].scale.max = float(ymax)

ResetZoomButton.on_click(resetZoom)
ResetZoomButton.layout.width = buttonWidth

l = link((zoom_interacts, 'value'), (fig, 'interaction'))

'''########Toolbar 2########'''
pz2 = PanZoom()
pzx2 = PanZoom()
pzy2 = PanZoom()

zoom_interacts2 = widgets.ToggleButtons(
                                        options=OrderedDict([
                                            ('xy ', pz2), 
                                            ('x ', pzx2), 
                                            ('y ', pzy2),   
                                            (' ', None)]),
                                            icons = ["arrows", "arrows-h", "arrows-v", "stop"],
                                            tooltips = ["zoom/pan em x & y", "zoom/pan em x apenas", "zoom/pan em y apenas", "cancela zoom/pan"]
                                        )
zoom_interacts2.index = 3
zoom_interacts2.style.button_width = buttonWidth
ResetZoomButton2 = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Reseta zoom',
    icon='refresh'
)

def resetZoom2(new):
    # Reset the x and y axes on the figure
    fig2.axes[0].scale.min = selected_start_date.value
    fig2.axes[1].scale.min = float(ymin)
    fig2.axes[0].scale.max = selected_end_date.value
    fig2.axes[1].scale.max = float(ymax)
    fig2.axes[2].scale.min = 0
    fig2.axes[2].scale.max = float(ymax2)

ResetZoomButton2.on_click(resetZoom2)
ResetZoomButton2.layout.width = buttonWidth

link((zoom_interacts2, 'value'), (fig2, 'interaction'))

'''########Toolbar 4########'''
pz4 = PanZoom()
pzx4 = PanZoom()
pzy4 = PanZoom()

zoom_interacts4 = widgets.ToggleButtons(
                                        options=OrderedDict([
                                            ('xy ', pz4), 
                                            ('x ', pzx4), 
                                            ('y ', pzy4),   
                                            (' ', None)]),
                                            icons = ["arrows", "arrows-h", "arrows-v", "stop"],
                                            tooltips = ["zoom/pan em x & y", "zoom/pan em x apenas", "zoom/pan em y apenas", "cancela zoom/pan"]
                                        )
zoom_interacts4.index = 3
zoom_interacts4.style.button_width = buttonWidth
ResetZoomButton4 = widgets.Button(
    description='',
    disabled=False,
    button_style='',
    tooltip='Reseta zoom',
    icon='refresh'
)

def resetZoom4(new):
    # Reset the x and y axes on the figure
    fig4.axes[0].scale.min = selected_start_date.value
    fig4.axes[1].scale.min = float(ymin)
    fig4.axes[0].scale.max = selected_end_date.value
    fig4.axes[1].scale.max = float(ymax)
    fig4.axes[2].scale.min = 0
    fig4.axes[2].scale.max = float(ymax4)

ResetZoomButton4.on_click(resetZoom4)
ResetZoomButton4.layout.width = buttonWidth

link((zoom_interacts4, 'value'), (fig4, 'interaction'))


'''###############################################################
   ####callback function to handle any change in the interface####
   ###############################################################'''
def build_forecast(ts,end_train_date):
    from statsmodels.tsa.arima.model import ARIMA
    from pmdarima.arima import auto_arima
    import io
    from contextlib import redirect_stdout
    ts_train = ts[ts.index<end_train_date]
    '''#############  Non-Seasonal ARIMA  ##############'''
    out = 'Performing Non-seasonal ARIMA Search:\n\n'
    f = io.StringIO()
    with redirect_stdout(f):
        model_auto_nonseasonal = auto_arima(ts_train, trace=True)
    out += f.getvalue()
    prediction_log.value = out
    order = model_auto_nonseasonal.get_params()['order']
    model_nonseasonal = ARIMA(ts_train, order=order)
    model_nonseasonal_fit = model_nonseasonal.fit()
    '''#############  Seasonal ARIMA  ##############'''
    out += '\nPerforming Seasonal ARIMA Search:\n\n'
    f = io.StringIO()
    with redirect_stdout(f):
        modelo_auto_seasonal = auto_arima(ts_train, m=13, trace=True)
    out += f.getvalue()
    prediction_log.value = out
    order2 = modelo_auto_seasonal.get_params()['order']
    seasonal_order = modelo_auto_seasonal.get_params()['seasonal_order']
    model_seasonal = ARIMA(ts_train, order=order2,seasonal_order=seasonal_order)
    model_seasonal_fit = model_seasonal.fit()
    if model_nonseasonal_fit.aic<=model_seasonal_fit.aic:
        ARIMA_model_fit = model_nonseasonal_fit
    else:
        ARIMA_model_fit = model_seasonal_fit
    out += '\nBest ARIMA Model is ' + ARIMA_model_fit.summary().tables[0].data[1][1]
    prediction_log.value = out
    
    
    
    '''############### Exponential Smoothing  ###########'''
    out += '\n\n\nPerforming Exponential Smoothing Search\n\n'
    prediction_log.value = out
    from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing
    fit1 = ExponentialSmoothing(ts_train).fit() #(N,N)
    out += 'Performing stepwise search to minimize aic\nETS(A,N,N) AIC=' + str(fit1.aic)
    prediction_log.value = out
    fit2 = ExponentialSmoothing(ts_train, trend=True).fit() #(A,N) Holt
    out += '\nETS(A,A,N) AIC=' + str(fit2.aic)
    prediction_log.value = out
    fit3 = ExponentialSmoothing(ts_train, trend=True, damped_trend=True).fit() #(Ad,N) Damped Holt
    out += '\nETS(A,Ad,N) AIC=' + str(fit3.aic)
    prediction_log.value = out
    fit4 = ExponentialSmoothing(ts_train, seasonal=13).fit() #(N,A) 
    out += '\nETS(A,N,A)[13] AIC=' + str(fit4.aic)
    prediction_log.value = out
    fit5 = ExponentialSmoothing(ts_train, trend=True, seasonal=13).fit() #(A,A) Holt Winter
    out += '\nETS(A,A,A)[13] AIC=' + str(fit5.aic)
    prediction_log.value = out
    fit6 = ExponentialSmoothing(ts_train, trend=True, damped_trend=True, seasonal=13).fit() #(A,A) Damped Holt Winter
    out += '\nETS(A,Ad,A)[13] AIC=' + str(fit6.aic)
    prediction_log.value = out
    models = [fit1, fit2, fit3, fit4, fit5, fit6]
    #Chooses model with lowest AIC (same criterium as auto_ARIMA)
    ETS_model_fit = models[np.argmin(np.array([m.aic for m in models]))] 
    out += '\nBest ETS Model is ' + ETS_model_fit.summary().tables[0].data[1][1]
    prediction_log.value = out
    out += '\nChoosing between best ARIMA and best ETS models.\nPerforming residual analysis:\n\n'
    prediction_log.value = out
    #Choose between ARIMA and ETS
    #Check redisudes serial correlation
    #Null hypothesis is no serial correlation
    ARIMA_resid_ljungbox_pvalues = ARIMA_model_fit.test_serial_correlation('ljungbox',lags=10)[0][1]
    ETS_resid_ljungbox_pvalues = ETS_model_fit.test_serial_correlation('ljungbox',lags=10)[0][1]
    #Check residues normality
    #Null hypothesis is normal distribution
    ARIMA_resid_normal_pvalue = ARIMA_model_fit.test_normality('jarquebera')[0][1]
    ETS_resid_normal_pvalue = ETS_model_fit.test_normality('jarquebera')[0][1]
    #Check residues heteroskedasticity
    #Null hypothesis is homoskedasticity
    ARIMA_resid_heteroskedasticity_pvalue = ARIMA_model_fit.test_heteroskedasticity('breakvar')[0][1]
    ETS_resid_heteroskedasticity_pvalue = ETS_model_fit.test_heteroskedasticity('breakvar')[0][1]
    
    if ((ARIMA_resid_ljungbox_pvalues >= 0.05).any()) & ((ETS_resid_ljungbox_pvalues < 0.05).any()):
        model_fit = ARIMA_model_fit
        model_name = ARIMA_model_fit.summary().tables[0].data[1][1]
        out += 'Residues from ARIMA passed serial correlation test!\nResidues from ETS failed serial correlation test!\n'
        prediction_log.value = out
    elif ((ARIMA_resid_ljungbox_pvalues < 0.05).any()) & ((ETS_resid_ljungbox_pvalues >= 0.05).any()):
        model_fit = ETS_model_fit
        model_name = ETS_model_fit.summary().tables[0].data[1][1]
        out += 'Residues from ETS serial correlation normality test!\nResidues from ARIMA failed serial correlation test!\n'
        prediction_log.value = out
    else:
        if ((ARIMA_resid_ljungbox_pvalues >= 0.05).any()) & ((ETS_resid_ljungbox_pvalues >= 0.05).any()):
            out += 'Both ARIMA and ETS residues passed serial correlation test!\n'
            prediction_log.value = out
        if ((ARIMA_resid_ljungbox_pvalues < 0.05).any()) & ((ETS_resid_ljungbox_pvalues < 0.05).any()):
            out += 'Both ARIMA and ETS residues failed serial correlation test!\n'
            prediction_log.value = out
            
        if (ARIMA_resid_normal_pvalue >= 0.05) & (ETS_resid_normal_pvalue < 0.05):
            model_fit = ARIMA_model_fit
            model_name = ARIMA_model_fit.summary().tables[0].data[1][1]
            out += 'Residues from ARIMA passed normality test!\nResidues from ETS failed normality test!\n'
            prediction_log.value = out
        elif (ARIMA_resid_normal_pvalue < 0.05) & (ETS_resid_normal_pvalue >= 0.05):
            model_fit = ETS_model_fit
            model_name = ETS_model_fit.summary().tables[0].data[1][1]
            out += 'Residues from ETS passed normality test!\nResidues from ARIMA failed normality test!\n'
            prediction_log.value = out
        else:
            if (ARIMA_resid_normal_pvalue >= 0.05) & (ETS_resid_normal_pvalue >= 0.05):
                out += 'Both ARIMA and ETS residues passed normality test!\n'
                prediction_log.value = out
            if (ARIMA_resid_normal_pvalue < 0.05) & (ETS_resid_normal_pvalue < 0.05):
                out += 'Both ARIMA and ETS residues failed normality test!\n'
                prediction_log.value = out
                
            if (ARIMA_resid_heteroskedasticity_pvalue >= 0.05) & (ETS_resid_heteroskedasticity_pvalue < 0.05):
                model_fit = ARIMA_model_fit
                model_name = ARIMA_model_fit.summary().tables[0].data[1][1]
                out += 'Residues from ARIMA passed heteroskedasticity test!\nResidues from ETS failed heteroskedasticity test!\n'
                prediction_log.value = out
            elif (ARIMA_resid_heteroskedasticity_pvalue < 0.05) & (ETS_resid_heteroskedasticity_pvalue >= 0.05):
                model_fit = ETS_model_fit
                model_name = ETS_model_fit.summary().tables[0].data[1][1]
                out += 'Residues from ETS passed heteroskedasticity test!\nResidues from ARIMA failed heteroskedasticity test!\n'
                prediction_log.value = out
            else:
                if (ARIMA_resid_heteroskedasticity_pvalue >= 0.05) & (ETS_resid_heteroskedasticity_pvalue >= 0.05):
                    out += 'Both ARIMA and ETS residues passed heteroskedasticity test!\n'
                    prediction_log.value = out
                if (ARIMA_resid_heteroskedasticity_pvalue < 0.05) & (ETS_resid_heteroskedasticity_pvalue < 0.05):
                    out += 'Both ARIMA and ETS residues failed heteroskedasticity test!\n'
                    prediction_log.value = out
                    
                    out += 'ARIMA SSE = ' + str(ARIMA_model_fit.sse) + '\nETS SSE = ' + str(ETS_model_fit.sse) + '\n'
                    prediction_log.value = out
                if ARIMA_model_fit.sse<=ETS_model_fit.sse:
                    model_fit = ARIMA_model_fit
                    model_name = ARIMA_model_fit.summary().tables[0].data[1][1]
                else:
                    model_fit = ETS_model_fit
                    model_name = ETS_model_fit.summary().tables[0].data[1][1]
    out += '### Final chosen model is ' + str(model_name) + ' ###'
    prediction_log.value = out
                
    return(model_fit,model_name)
clicked_predict=False
def on_click_predict(b):
    global clicked_predict
    clicked_predict = True
    on_change(b)

def clear_figure(fig,tb):
    fig.axes = []
    fig.marks = []
    fig.title = ''
    tb.children = ()
    
def on_change(b):
    error_label.value = ''
    import pandas as pd
    from datetime import date
    if tab.selected_index==0:
        help_text.value = help_text1
    elif tab.selected_index==1:
        help_text.value = help_text2
    else:
        help_text.value = help_text3
    if selected_start_date.disabled==False:
        if not selected_equips.options:
            error_msg = 'Por favor selecione um equipamento!'
            if tab.selected_index==0:
                clear_figure(fig,tb)
            elif tab.selected_index==1:
                clear_figure(fig2,tb2)
                clear_figure(fig3,tb2)
            else:
                clear_figure(fig4,tb4)
            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
        elif selected_start_date.value==None:
            error_msg = 'Por favor selecione uma data inicial!'
            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
        elif selected_end_date.value==None:
            error_msg = 'Por favor selecione uma data final!'
            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
        elif selected_start_date.value>=selected_end_date.value:
            error_msg = 'Erro! Data inicial deve ser menor que data final!'
            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
            return(0)
            selected_start_date.value = None
        elif pd.to_datetime(selected_end_date.value)>pd.to_datetime(date.today()):
            error_msg = 'Erro! Data final deve ser menor que data atual!'
            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
            return(0)
            selected_end_date.value = None
        else:
            import numpy as np
            import pandas as pd
            global mask1, mask2, start_date, end_date, x_data1, y_data1, x_data2, y_data2, x_data3, y_data3, x_data
            bar2.selected = []
            bar3.selected = []
            bar.selected = []
            fig3.axes = []
            fig3.marks = []
            fig3.title = ''
            global ymin, ymax, ymax2, clicked_predict, reset_prediction, break_ts
            start_date = pd.to_datetime(selected_start_date.value)
            #last second of the day
            end_date = pd.to_datetime(selected_end_date.value) + pd.to_timedelta(1,'D') - pd.to_timedelta(1,'S')
            if tab.selected_index==0:
                x_data1,y_data1,x_data2,y_data2,break_ts = update_data(selected_equips.options,start_date,end_date)
                first_record = x_data1[0]
                if len(break_ts.index)>=50:
                    fiftieth_closed_OS_record = break_ts.index[49]
                else:
                    fiftieth_closed_OS_record = None

                if tendency.value==True:
                    label6.layout.visibility='visible'
                    label7.layout.visibility='visible'
                    label8.layout.visibility='visible'
                    prediction_range.layout.visibility='visible'
                    selected_tendency_start_date.layout.visibility='visible'
                    predict_button.layout.visibility='visible'

                    if (selected_tendency_start_date.value!=None) & (clicked_predict==True):
                        from datetime import date
                        today = pd.to_datetime(date.today())
                        prediction_start_date = pd.to_datetime(selected_tendency_start_date.value)

                        if (prediction_start_date>today):
                            selected_tendency_start_date.value = None
                            error_msg = 'Erro! Data inicial da previsao nao pode ser maior que hoje.'
                            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
                        elif prediction_start_date<start_date:
                            error_msg = 'Erro! Data inicial da previsao menor que data inicial do grafico.'
                            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
                        elif fiftieth_closed_OS_record==None:
                            error_msg = 'Erro! Dados insuficientes para gerar serie temporal!'
                            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
                        elif (prediction_start_date-pd.Timedelta(prediction_range.value,'W'))<fiftieth_closed_OS_record:
                            error_msg = 'Erro! Dados insuficientes para gerar serie temporal! \n Tente uma data de previsao mais recente.'
                            error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
                        else:
                            global yo2, xo2, yo1, xo1, xo3, yo3, ts3, ts1, prediction, conf_int, ts2
                            
                            
                            ts2 = pd.Series(y_data2, index=x_data2)
                            ts2 = ts2.groupby(pd.Grouper(freq="W")).last().fillna(method='ffill')
                            xo2 = ts2.index
                            yo2 = ts2.values
                            x_data2 = xo2 #temp
                            y_data2 = yo2 #temp
                            ts1 = pd.Series(y_data1, index=x_data1)
                            ts1 = ts1.groupby(pd.Grouper(freq="W")).last().fillna(method='ffill')
                            xo1 = ts1.index
                            yo1 = ts1.values
                            if break_ts.empty==False:
                                error_msg = 'Buscando melhor modelo... Por favor aguarde...'
                                error_label.value = f"<b><font color='orange'>{error_msg}</b>" #r'\(\color{orange} {' + error_msg + '}\)'
                                log_box.layout.visibility = 'visible'
                                ts3 = break_ts.groupby(pd.Grouper(freq="W")).last().fillna(method='ffill')
                                xo3 = ts3.index
                                yo3 = ts3.values
                                global prediction_end, prediciton_starting_week, prediction_start
                                global prediction_starting_week, forecast, conf_int
                                #convert to timestamp
                                tendency_start_date = pd.Timestamp(selected_tendency_start_date.value)
                                
                                #arranges prediction start to a sunday (since working with weeks)
                                prediction_starting_week = tendency_start_date + pd.to_timedelta(6-tendency_start_date.dayofweek, unit='d')# - pd.Timedelta(1,'W')
                                #build a model using training data previous to tendency_start_date

                                model_fit,model_name = build_forecast(ts3,prediction_starting_week)
                                # predict values from built model for (at first) q year (52weeks) before and after tendency_start_date
                                time_delta = pd.to_timedelta(int(prediction_range.value),'W')

                                if ts1.index[-1]<prediction_starting_week:
                                    ts1_comp_idx = pd.date_range(ts1.index[-1],prediction_starting_week,freq='W')[1:]
                                    ts1_comp = pd.Series(np.ones((len(ts1_comp_idx)))*ts1.values[-1], index=ts1_comp_idx)
                                    ts1 = ts1.append(ts1_comp)
                                
                                prediction_start = prediction_starting_week
                                prediction_end = prediction_starting_week
                                prediction_start -= time_delta
                                prediction_end += time_delta
                                # make predictions (using ".predict" here because it can generate in_sample data)
                                prediction = model_fit.predict(prediction_start.strftime('%Y-%m-%d'),
                                                               prediction_end.strftime('%Y-%m-%d'), dynamic=False)
        
                                #Find index of ts1 (blue curve) where it macthes tendency_start_date (rounded to week sunday)
                                idx_ts1 = np.argwhere(ts1.index==prediction_starting_week)[0][0]
                                #Find same index of prediction vector 
                                idx_prediction = np.argwhere(prediction.index==prediction_starting_week)[0][0]
                                ### Arrange prediction (based on break rate) to match orange curve (available equips)###
                                # Prediction values until the end of ts1 receive corresponding ts1 values minus prediction values 
                                prediction.values[:idx_prediction] = ts1.values[idx_ts1-idx_prediction:idx_ts1] - prediction.values[:idx_prediction]
                                # Prediction values beyond ts1 receive ts1 "last value" (equivalent to tendency_start_date) minus prediction values
                                prediction.values[idx_prediction:] = ts1.values[idx_ts1] - prediction.values[idx_prediction:]

                                #if the forecast start already has data
                                if prediction_starting_week<=ts3.index[-1]:
                                    # number of weeks to forecast conf intervals
                                    n_forecast = int(prediction_range.value)+1
                                    n_forecast = int((prediction_end-prediction_starting_week).days/7)+1
                                    forecast_insample = prediction[prediction.index<prediction_starting_week]
                                    forecast_outsample = prediction[prediction.index>=prediction_starting_week]
                                else: #otherwise the number of weeks to predict should be increased to match the last date
                                    n_forecast = int((prediction_end-ts3.index[-1]).days/7)
                                    forecast_insample = prediction[prediction.index<=ts3.index[-1]]
                                    forecast_outsample = prediction[prediction.index>ts3.index[-1]]
                                 # generate forecast (here we use ".forecast" instead of ".predict" because it can generate confidence intervals)
                                forecast = model_fit.get_forecast(n_forecast)
                                # get conf intervals for generated forecast
                                conf_int = forecast.conf_int()
                                # Arrnage confidence intervals the same way as predictions to match available equips curve (orange curve)
                                conf_int = ts1.values[idx_ts1] - conf_int
                                # if the start of the prediction preceeds the forecasts conf_int: slice con_int to match prediction line
                                if prediction_start>conf_int.index[0]:
                                    conf_int = conf_int[conf_int.index>=prediction_start]
                                # if in_sample prediction has dates that preceed plot interval: slice in_sample plot
                                if not forecast_insample.empty:
                                    if forecast_insample.index[0]<start_date:
                                        forecast_insample = forecast_insample[forecast_insample.index>=start_date]
                                
                                line_conf_int.x = conf_int.index
                                line_conf_int.y = [conf_int.iloc[:,0].values,conf_int.iloc[:,1].values]
                                line_forecast.x = forecast_outsample.index
                                line_forecast.y = forecast_outsample.values
                                scatter_pred_start_point.x = np.array([prediction_starting_week.to_numpy()])
                                scatter_pred_start_point.y = prediction[prediction.index==prediction_starting_week].values
                                
                                line_forecast_in_sample.x = forecast_insample.index
                                line_forecast_in_sample.y = forecast_insample.values
                                line_forecast_in_sample.labels = [model_name + ' Predicao sobre dados']
                                line_forecast.labels = [model_name + ' Previsao futura (IC95%)']
                                error_msg = ''
                                error_label.value = error_msg
    
                else:
                    label6.layout.visibility='hidden'
                    label7.layout.visibility='hidden'
                    label8.layout.visibility='hidden'
                    prediction_range.layout.visibility='hidden'
                    selected_tendency_start_date.layout.visibility='hidden'
                    predict_button.layout.visibility='hidden'
                    log_box.layout.visibility = 'hidden'
                    clicked_predict = False
                    selected_tendency_start_date.value=None
                ###slices data to lay between start and end dates##
                
                mask1 = (x_data1>=start_date)&(x_data1<=end_date)
                mask2 = (x_data2>=start_date)&(x_data2<=end_date)

                # if both curves have data points inside interval, slice and fill edges
                if mask1.any():
                    x_data1 = x_data1[mask1]
                    y_data1 = y_data1[mask1]
               
                    #add last point
                    x_data1 = np.append(x_data1,end_date.to_numpy())
                    x_data1 = pd.to_datetime(x_data1)
                    y_data1 = np.append(y_data1,y_data1[-1])
            
                    # add first point as 0
                    x_data1 = np.insert(x_data1,0,start_date.to_numpy())
                    if start_date>first_record:
                        y_data1 = np.insert(y_data1,0,y_data1[0])
                    else:
                        y_data1 = np.insert(y_data1,0,0)
                else:
                    # if curve1 has no data points inside interval
                    # look if whole data is before selected interval
                    if (x_data1[0]<start_date) & (x_data1[-1]<start_date):
                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data1 = np.array([y_data1[-1],y_data1[-1]])
                    # look if whole data is after selected interval
                    elif (x_data1[0]>start_date) & (x_data1[-1]>start_date):
                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data1 = np.array([0,0])
                    # look is part of the data is before start, part is after, but none is between
                    else:
                        y_data1 = np.array([y_data1[x_data1<start_date.to_numpy()][-1], y_data1[x_data1>end_date.to_numpy()][0]])
                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        
                # if both curves have data points inside interval, slice and fill edges
                if mask2.any():
                    x_data2 = x_data2[mask2]
                    y_data2 = y_data2[mask2]
                    #add last point
                    x_data2 = np.append(x_data2,end_date.to_numpy())
                    x_data2 = pd.to_datetime(x_data2)
                    y_data2 = np.append(y_data2,y_data2[-1])
                    # add first point as 0
                    x_data2 = np.insert(x_data2,0,start_date.to_numpy())
                    if start_date>first_record:
                        y_data2 = np.insert(y_data2,0,y_data2[0])
                    else:
                        y_data2 = np.insert(y_data2,0,0)
                else:
                    # if curve1 has no data points inside interval
                    # look if whole data is before selected interval
                    if (x_data2[0]<start_date) & (x_data2[-1]<start_date):
                        x_data2 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data2 = np.array([y_data2[-1],y_data2[-1]])
                    # look if whole data is after selected interval
                    elif (x_data2[0]>start_date) & (x_data2[-1]>start_date):
                        x_data2 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data2 = np.array([0,0])
                    # look is part of the data is before start, part is after, but none is between
                    else:
                        x_data2 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data2 = np.array([y_data2[x_data2<start_date.to_numpy()][-1], y_data2[x_data2>end_date.to_numpy()][0]])

                ymin = min(np.amin(y_data1),np.amin(y_data2))
                ymax = max(np.amax(y_data1),np.amax(y_data2))
                    ##############################
                with scatter1.hold_sync():
                    scatter1.x = x_data1
                    scatter1.y = y_data1
                with scatter1_edges.hold_sync():
                    scatter1_edges.x = np.array([x_data1[0],x_data1[-1]])
                    scatter1_edges.y = np.array([y_data1[0],y_data1[-1]])
                with line1.hold_sync():
                    line1.x = x_data1
                    line1.y = y_data1
                    line1.fill = 'bottom'
                    line1.marker = 'square'
                    line1.marker_size = 20
                    line1.fill_opacities = [0.2]
                with scatter2.hold_sync():
                    scatter2.x = x_data2
                    scatter2.y = y_data2
                with line2.hold_sync():
                    line2.x = x_data2
                    line2.y = y_data2
                    line2.fill = 'bottom'
                    line2.marker = 'circle'
                    line2.fill_opacities = [0.2]
                    line2.marker_size = 8
                    line2.opacities = [.5]
                x_dt.min = pd.to_datetime(selected_start_date.value)
                x_dt.max = pd.to_datetime(selected_end_date.value)
                if (pd.to_datetime(selected_end_date.value) - pd.to_datetime(selected_start_date.value)).days<240:
                    ax_x.tick_format = "%d-%b-%y"
                else:
                    ax_x.tick_format = "%b-%y"
                with y_sc1.hold_sync():
                    y_sc1.min = np.around(0.9*ymin)
                    y_sc1.max = np.around(1.1*ymax)
                with ax_y1.hold_sync():
                    ax_y1.label_color='black'
                    ax_y1.color='black'
                    ax_y1.tick_style={'font-size': 14}
                    ax_y1.grid_color='black' 
                if (tendency.value==True) & (break_ts.empty==False):
                    if (selected_tendency_start_date.value!=None) & (clicked_predict==True):
                        fig.marks = [line1,line2,line_forecast_in_sample,line_forecast,line_conf_int,scatter1_edges,scatter2_edges,scatter1,scatter2,scatter_pred_start_point]
                    else:
                        fig.marks = [line1,line2,scatter1_edges,scatter2_edges,scatter1,scatter2]
                else:
                    fig.marks = [line1,line2,scatter1_edges,scatter2_edges,scatter1,scatter2]
                fig.axes = [ax_x, ax_y1]
                fig.legend_location = 'top-left'
                pz.scales={'x': [x_dt], 'y': [y_sc1]}
                pzx.scales={'x': [x_dt]}
                pzy.scales={'y': [y_sc1], }
                tb.children = ()
                tb.children += (widgets.HBox([zoom_interacts,ResetZoomButton,save_data_button,save_fig_button]),)
                tb.layout.justify_content = 'center' 
                clicked_predict = False
            elif tab.selected_index==1:
                x_data1,y_data1,x_data2,y_data2,x_data3,y_data3,break_ts = update_data2(selected_equips.options,
                                                                                               selected_start_date.value,
                                                                                               selected_end_date.value)
                first_record = x_data1[0]
                mask1 = (x_data1>=start_date)&(x_data1<=end_date)
                mask2 = (x_data2>=start_date)&(x_data2<=end_date)
                mask3 = (x_data3>=start_date)&(x_data3<=end_date)
                # if curve1 has data points inside interval, slice and fill edges
                if mask1.any():
                    x_data1 = x_data1[mask1]
                    y_data1 = y_data1[mask1]
                    #add last point
                    x_data1 = np.append(x_data1,end_date.to_numpy())
                    x_data1 = pd.to_datetime(x_data1)
                    y_data1 = np.append(y_data1,y_data1[-1])
                    # add first point as 0
                    x_data1 = np.insert(x_data1,0,start_date.to_numpy())
                    if start_date>first_record:
                        y_data1 = np.insert(y_data1,0,y_data1[0])
                    else:
                        y_data1 = np.insert(y_data1,0,0)
                else:
                    # if curve1 has no data points inside interval
                    # look if whole data is before selected interval
                    if (x_data1[0]<start_date) & (x_data1[-1]<start_date):
                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data1 = np.array([y_data1[-1],y_data1[-1]])
                    # look if whole data is after selected interval
                    elif (x_data1[0]>start_date) & (x_data1[-1]>start_date):
                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data1 = np.array([0,0])
                    # look is part of the data is before start, part is after, but none is between
                    else:
                        y_data1 = np.array([y_data1[x_data1<start_date.to_numpy()][-1], y_data1[x_data1>end_date.to_numpy()][0]])
                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                
                x_data2 = x_data2[mask2]
                y_data2 = y_data2[mask2]
                x_data3 = x_data3[mask3]
                y_data3 = y_data3[mask3]
                ymin = np.amin(y_data1)
                ymax = np.amax(y_data1)

                # if any data is empty, fill with zeros the other one
                # if both have data, built time array comprising both data, fill with zeros missing y values
                if (len(x_data2)==0) & (len(x_data3)!=0):
                    x_data = x_data3
                    y_data2 = np.zeros_like(y_data3)
                elif (len(x_data3)==0) & (len(x_data2)!=0):
                    x_data = x_data2
                    y_data3 = np.zeros_like(y_data2)
                elif (len(x_data2)!=0) & (len(x_data3)!=0):
                    earliest_date = np.amin(np.concatenate([x_data2, x_data3]))
                    latest_date = np.amax(np.concatenate([x_data2, x_data3]))
                    x_data = pd.date_range(earliest_date,latest_date,freq='M').values
                    y_data22 = np.zeros(len(x_data))
                    y_data33 = np.zeros(len(x_data))
                    y_data22[np.isin(x_data,x_data2)] = y_data2
                    y_data33[np.isin(x_data,x_data3)] = y_data3
                    y_data2 = y_data22
                    y_data3 = y_data33
                else:
                    x_data = x_data2
                # Transforms dates from month_end to the middle of the month, so bars are centered inside each month
                x_data = (pd.to_datetime(pd.to_datetime(x_data).strftime("%Y-%m")) + pd.Timedelta(2,'W')).to_numpy()
                
                if len(y_data2)==0:
                    ymax2 = 0
                else:
                    ymax2 = np.amax(y_data2)
                if len(y_data3)==0:
                    ymax3 = 0
                else:
                    ymax3 = np.amax(y_data3)
                
                with scatter1.hold_sync():
                    scatter1.x = x_data1
                    scatter1.y = y_data1
                with scatter1_edges.hold_sync():
                    scatter1_edges.x = np.array([x_data1[0],x_data1[-1]])
                    scatter1_edges.y = np.array([y_data1[0],y_data1[-1]])
                with line1.hold_sync():
                    line1.x = x_data1
                    line1.y = y_data1
                    line1.fill = 'bottom'
                    line1.marker = 'square'
                    line1.marker_size = 20
                    line1.fill_opacities = [0.2]

                if selected_analysis.value==1:
                    with bar2.hold_sync():
                        bar2.x = x_data
                        bar2.y = y_data2
                        bar2.colors = ['seagreen']
                        bar2.interactions = {
                            'legend_hover': 'highlight_axes',
                            'hover': 'tooltip',
                            'click':'select'
                        }
                        bar2.selected_style={'opacity': 1.0, 'stroke': 'Red'}
                        bar2.unselected_style={'opacity': 0.5}
                    fig2.marks = [line1,bar2,scatter1_edges,scatter1]
                elif selected_analysis.value==2:
                    with bar3.hold_sync():
                        bar3.x = x_data
                        bar3.y = y_data3
                        bar3.colors = ['yellowgreen']
                        bar3.interactions = {
                            'legend_hover': 'highlight_axes',
                            'hover': 'tooltip',
                            'click':'select'
                        }
                        bar3.selected_style={'opacity': 1.0, 'stroke': 'Red'}
                        bar3.unselected_style={'opacity': 0.5}
                    ymax2 = ymax3
                    fig2.marks = [line1,bar3,scatter1_edges,scatter1]
                else:
                    with bar.hold_sync():
                        bar.x = x_data
                        bar.y = [y_data2, y_data3]
                        bar.interactions = {
                            'legend_hover': 'highlight_axes',
                            'hover': 'tooltip',
                            'click':'select'
                        }
                        bar.selected_style={'opacity': 1.0, 'stroke': 'Red'}
                        bar.unselected_style={'opacity': 0.5}
                    ymax2 = max(ymax2,ymax3)
                    fig2.marks = [line1,bar,scatter1_edges,scatter1]

                ax_y2.label=selected_analysis.label
                with ax_y1.hold_sync():
                    ax_y1.label_color='steelblue'
                    ax_y1.color='steelblue'
                    ax_y1.tick_style={'stroke': 'steelblue', 'font-size': 12}
                    ax_y1.grid_color='steelblue'

                with x_dt.hold_sync():
                    x_dt.min = pd.to_datetime(selected_start_date.value)
                    x_dt.max = pd.to_datetime(selected_end_date.value)
                if (pd.to_datetime(selected_end_date.value) - pd.to_datetime(selected_start_date.value)).days<240:
                    ax_x.tick_format = "%d-%b-%y"
                else:
                    ax_x.tick_format = "%b-%y"
                with y_sc1.hold_sync():
                    y_sc1.min = np.around(0.9*ymin)
                    y_sc1.max = np.around(1.1*ymax)
                with y_sc2.hold_sync():
                    y_sc2.min = 0
                    y_sc2.max = 1.1*ymax2   
                fig2.axes = [ax_x,ax_y1,ax_y2]
                fig2.legend_location = 'top-left'
                pz2.scales={'x': [x_dt], 'y': [y_sc1,y_sc2]}
                pzx2.scales={'x': [x_dt]}
                pzy2.scales={'y': [y_sc1,y_sc2], }
                tb2.children = ()
                tb2.children += (widgets.HBox([zoom_interacts2,ResetZoomButton2,save_data_button,save_fig_button2]),)
                tb2.layout.justify_content = 'center'

            elif tab.selected_index==2:
                x_data1,y_data1,x_data2,y_data2,duration_ts = update_data3(selected_equips.options)
                first_record = x_data1[0]
                duration_ts = duration_ts.groupby(pd.Grouper(freq="M")).median().fillna(0)
                xo4 = duration_ts.index
                yo4= duration_ts.values
                x_data2 = xo4#temp
                y_data2 = yo4#temp
                ####    DATE MASK AND FILL EDGES####
                ###slices data to lay between start and end dates##
                
                mask1 = (x_data1>=start_date)&(x_data1<=end_date)
                mask2 = (x_data2>=start_date)&(x_data2<=end_date)

                # if blue curve1 has data points inside interval, slice and fill edges
                if mask1.any():

                    x_data1 = x_data1[mask1]
                    y_data1 = y_data1[mask1]

                    #add last point
                    x_data1 = np.append(x_data1,end_date.to_numpy())
                    x_data1 = pd.to_datetime(x_data1)
                    y_data1 = np.append(y_data1,y_data1[-1])
                

                    x_data1 = np.insert(x_data1,0,start_date.to_numpy())
                    if start_date>first_record:
                        # repeat first point at start time
                        y_data1 = np.insert(y_data1,0,y_data1[0])
           
                    else:
                        # add first point as 0
                        y_data1 = np.insert(y_data1,0,0)
               
                else:
                    # if curve1 has no data points inside interval
                    # look if whole data is before selected interval
                    if (x_data1[0]<start_date) & (x_data1[-1]<start_date):

                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data1 = np.array([y_data1[-1],y_data1[-1]])
                    # look if whole data is after selected interval
                    elif (x_data1[0]>start_date) & (x_data1[-1]>start_date):

                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                        y_data1 = np.array([0,0])
                    # look is part of the data is before start, part is after, but none is between
                    else:

                        y_data1 = np.array([y_data1[x_data1<start_date.to_numpy()][-1], y_data1[x_data1>end_date.to_numpy()][0]])
                        x_data1 = np.array([start_date.to_numpy(),end_date.to_numpy()])
                
            
                x_data2 = x_data2[mask2]
                y_data2 = y_data2[mask2]

                ########################################################################
                # Transforms dates from month_end to the middle of the month, so bars are centered inside each month
                x_data2 = (pd.to_datetime(pd.to_datetime(x_data2).strftime("%Y-%m")) + pd.Timedelta(2,'W')).to_numpy()
                ymin = np.amin(y_data1)
                ymax = np.amax(y_data1)

                if len(y_data2)==0:
                    ymax4 = 0
                else:
                    ymax4 = np.amax(y_data2)

                with scatter1.hold_sync():
                    scatter1.x = x_data1
                    scatter1.y = y_data1
                with scatter1_edges.hold_sync():
                    scatter1_edges.x = np.array([x_data1[0],x_data1[-1]])
                    scatter1_edges.y = np.array([y_data1[0],y_data1[-1]])
                with line1.hold_sync():
                    line1.x = x_data1
                    line1.y = y_data1
                    line1.fill = 'bottom'
                    line1.marker = 'square'
                    line1.marker_size = 20
                    line1.fill_opacities = [0.2]

                with line4.hold_sync():
                    line4.x = x_data2
                    line4.y = y_data2
 
                x_dt.min = pd.to_datetime(selected_start_date.value)
                x_dt.max = pd.to_datetime(selected_end_date.value)
                if (pd.to_datetime(selected_end_date.value) - pd.to_datetime(selected_start_date.value)).days<240:
                    ax_x.tick_format = "%d-%b-%y"
                else:
                    ax_x.tick_format = "%b-%y"
                with y_sc1.hold_sync():
                    y_sc1.min = np.around(0.9*ymin)
                    y_sc1.max = np.around(1.1*ymax)
                with y_sc4.hold_sync():
                    y_sc4.min = 0
                    y_sc4.max = 1.1*ymax4
                with ax_y1.hold_sync():
                    ax_y1.label_color='steelblue'
                    ax_y1.color='steelblue'
                    ax_y1.tick_style={'stroke': 'steelblue', 'font-size': 12}
                    ax_y1.grid_color='steelblue'
                fig4.marks = [line1,line4,scatter1_edges,scatter1]#,scatter4]
                fig4.axes = [ax_x, ax_y1, ax_y4]
                fig4.legend_location = 'top-left'
                pz4.scales={'x': [x_dt], 'y': [y_sc1,y_sc4]}
                pzx4.scales={'x': [x_dt]}
                pzy4.scales={'y': [y_sc1,y_sc4], }
                tb4.children = ()
                tb4.children += (widgets.HBox([zoom_interacts4,ResetZoomButton4,save_data_button,save_fig_button]),)
                tb4.layout.justify_content = 'center' 
    
'''######################################
   ####Functions to save data and fig####
   ######################################'''
# Opens filechooser interface
def on_save_data(b):
    error_label.value = ''
    if selected_start_date.value==None:
        error_msg = 'Por favor selecione uma data inicial!'
        error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
    elif selected_end_date.value==None:
        error_msg = 'Por favor selecione uma data final!'
        error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
    elif not selected_equips.options:
        error_msg = 'Por favor selecione um equipamento!'
        error_label.value = f"<b><font color='red'>{error_msg}</b>" #r'\(\color{red} {' + error_msg + '}\)'
    else:
        save_box.layout.visibility = 'visible'
# Saves figure
def on_save_figure(b):
    import time
    error_label.value = ''
    if len(selected_equips.options)!=0:
        if len(selected_equips.options)==1:
            string_equips = selected_equips.options[0]
        else:
            string_equips = 'Conjunto de Equipamentos'
    if tab.selected_index==0:
        fig_file_name = string_equips + ' Ativos DE ' + selected_start_date.value.strftime('%Y-%m-%d') + ' A ' + selected_end_date.value.strftime('%Y-%m-%d')
        fig.save_png(fig_file_name)
    elif tab.selected_index==1:
        fig_file_name = string_equips + ' Custos DE ' + selected_start_date.value.strftime('%Y-%m-%d') + ' A ' + selected_end_date.value.strftime('%Y-%m-%d')
        fig2.save_png(fig_file_name)
        if fig3.title!='':
            time.sleep(0.5)
            fig_file_name = string_equips + '_' + fig3.title
            fig3.save_png(fig_file_name)
    elif tab.selected_index==2:
        fig_file_name = string_equips + ' Duracao de OS DE ' + selected_start_date.value.strftime('%Y-%m-%d') + ' A ' + selected_end_date.value.strftime('%Y-%m-%d')
        fig4.save_png(fig_file_name)

def append_asterisks(df):
    # adds '*' after first data points (to indicate they are replicates of the closest true value)
    if df.shape[1]>2:
        df.iloc[0,1] = str(df.iloc[0,1]) + '*'
    df.iloc[0,-1] = str(df.iloc[0,-1]) + '*'
    # adds '*' after last data points (to indicate they are replicates of the closest true value)
    if df.shape[1]>2:
        valid_values = pd.notna(df.iloc[:,1])
        last_valid_line = valid_values[valid_values>0].index[-1]
        df.iloc[last_valid_line,1] = str(df.iloc[last_valid_line,1]) + '*'

    valid_values = pd.notna(df.iloc[:,-1])
    last_valid_line = valid_values[valid_values>0].index[-1]
    df.iloc[last_valid_line,-1] = str(df.iloc[last_valid_line,-1]) + '*'
    return(df)        

# Save whole data in specified directory with chosen format    
def on_save(b):
    import time
    error_label.value = ''
#     if fc.selected==None:
#         save_msg = 'Por \ favor \ selecione \ uma \ pasta!'
#         save_label.value = r'\(\color{red} {' + save_msg + '}\)'
#     else:
    try:
        global ymin, ymax, ymax2
        global x_data1, y_data1, x_data2, y_data2, x_data3, y_data3
        if len(selected_equips.options)!=0:
            if len(selected_equips.options)==1:
                string_equips = selected_equips.options[0]
            else:
                string_equips = 'Conjunto de Equipamentos'
        if tab.selected_index==0:
            d = {'Data': x_data2, 'Disponivel': y_data2}
            df1 = pd.DataFrame(data=d)
            d = {'Data2': x_data1, 'Ativo': y_data1}
            df2 = pd.DataFrame(data=d)
            df = pd.concat([df1,df2],axis=1)
            data_file_name = string_equips + ' Ativos DE ' + selected_start_date.value.strftime('%Y-%m-%d') + ' A ' + selected_end_date.value.strftime('%Y-%m-%d')
            filepath = os.path.join(data_folder,data_file_name)
            df = append_asterisks(df)
            if selected_file_type.value==1:
                file = filepath + '.csv'
                df.to_csv(file, index=False)
            elif selected_file_type.value==2:
                file = filepath + '.xls'
                df.to_excel(file, index=False)
            else:
                file = filepath + '.txt'
                df.loc[:,'Data'] = df.loc[:,'Data'].dt.strftime('%d/%m/%Y')
                df.loc[:,'Data2'] = df.loc[:,'Data2'].dt.strftime('%d/%m/%Y')
                df.fillna('', inplace=True)
                np.savetxt(file, df.values, fmt='%-8s',delimiter='\t',header='Data\t\tDisponivel\tData2\t\tAtivo')

        elif tab.selected_index==1:
            d = {'Data': x_data1, 'Ativo': y_data1}
            df1 = pd.DataFrame(data=d)
            df1 = append_asterisks(df1)
            d = {'Data2': x_data2, 'Custo de Material': np.around(y_data2,2)}
            df2 = pd.DataFrame(data=d)
            d = {'Data3': x_data3, 'Custo de Servico Externo': np.around(y_data3,2)}
            df3 = pd.DataFrame(data=d)
            df = pd.concat([df1,df2,df3],axis=1)
            data_file_name = string_equips + ' Custos DE ' + selected_start_date.value.strftime('%Y-%m-%d') + ' A ' + selected_end_date.value.strftime('%Y-%m-%d')
            filepath = os.path.join(data_folder,data_file_name)

            if selected_file_type.value==1:
                file = filepath + '.csv'
                df.to_csv(file, index=False)
            elif selected_file_type.value==2:
                file = filepath + '.xls'
                df.to_excel(file, index=False)
            else:
                file = filepath + '.txt'
                df.loc[:,'Data'] = df.loc[:,'Data'].dt.strftime('%d/%m/%Y')
                df.loc[:,'Data2'] = df.loc[:,'Data2'].dt.strftime('%d/%m/%Y')
                df.loc[:,'Data3'] = df.loc[:,'Data3'].dt.strftime('%d/%m/%Y')
                df.fillna('', inplace=True)
                np.savetxt(file, df.values, fmt='%-8s',delimiter='\t',header='Data\t\tAtivo\t\tData2\t\tCusto de Mat.\tData3\t\tCusto de Servico Externo')
        elif tab.selected_index==2:
            d = {'Data': x_data1, 'Ativo': y_data1}
            df1 = pd.DataFrame(data=d)
            df1 = append_asterisks(df1)
            d = {'Data2': x_data2, 'Mediana da Duracao (horas)': np.around(y_data2,2)}
            df2 = pd.DataFrame(data=d)
            df = pd.concat([df1,df2],axis=1)
            data_file_name = string_equips + ' Duracao de OS DE ' + selected_start_date.value.strftime('%Y-%m-%d') + ' A ' + selected_end_date.value.strftime('%Y-%m-%d')
            filepath = os.path.join(data_folder,data_file_name)

            if selected_file_type.value==1:
                file = filepath + '.csv'
                df.to_csv(file, index=False)
            elif selected_file_type.value==2:
                file = filepath + '.xls'
                df.to_excel(file, index=False)
            else:
                file = filepath + '.txt'
                df.loc[:,'Data'] = df.loc[:,'Data'].dt.strftime('%d/%m/%Y')
                df.loc[:,'Data2'] = df.loc[:,'Data2'].dt.strftime('%d/%m/%Y')
                df.fillna('', inplace=True)
                np.savetxt(file, df.values, fmt='%-8s',delimiter='\t',header='Data\t\tAtivo\t\tData2\t\tMediana da Duracao (horas)')    
        save_msg = 'Dados salvos em ' + data_folder
        save_label.value = f"<b><font color='green'>{save_msg}</b>"# r'\(\color{green} {' + save_msg + '}\)'
        time.sleep(4)
        save_msg=''
        save_label.value = save_msg
        save_box.layout.visibility = 'hidden'
    except PermissionError:
        save_msg = 'Erro! Arquivo aberto! Feche o arquivo para sobrescrever.'
        save_label.value = f"<b><font color='red'>{save_msg}</b>" #r'\(\color{red} {' + save_msg + '}\)'

def on_add(change):
    if change['new']!='':
        if len(selected_equips.options)==0:
            options = [change['new']]
        else:
            options = list(selected_equips.options)
            options.append(change['new'])
        if change['new'] not in selected_equips.options:
            selected_equips.options = options
            on_change(0)
        add_equips.value = ''
def on_del(change):
    if len(change['new']) == 1:
        options = list(change['owner'].options)
        index = change['owner'].index[0]
#         add_equips.options.append(options[index])
#         add_equips.options.sort()
        del options[index]
        change['owner'].options = options
        
        on_change(0)
        
'''######################################
   #######Link widgets to callbacks######
   ######################################'''
selected_analysis.observe(on_change,names='value')
add_equips.observe(on_add, names=['value'])
selected_equips.observe(on_del,names=['value'])

selected_start_date.observe(on_change,names='value')
selected_end_date.observe(on_change,names='value')
tab.observe(on_change,names='selected_index')
save_data_button.on_click(on_save_data)
save_button.on_click(on_save)
save_fig_button.on_click(on_save_figure)
save_fig_button2.on_click(on_save_figure)
tendency.observe(on_change,type='change')
selected_tendency_start_date.observe(on_change,names='value')
prediction_range.observe(on_change,names='value')
predict_button.on_click(on_click_predict)


'''#################################
   #######Build main interface######
   #################################'''
vbox = widgets.VBox([label2,
              add_equips,
              selected_equips,
              label3,
              selected_start_date,
              label4,
              selected_end_date,
              tendency,
              label6,
              selected_tendency_start_date,
              widgets.HBox([label7,prediction_range,label8],layout=widgets.Layout(align_self='flex-start')),
              predict_button,
              error_label,
              log_box
             ])

vbox2 = widgets.VBox([label1,
              selected_analysis,
              label2,
              add_equips,
              selected_equips,
              label3,
              selected_start_date,
              label4,
              selected_end_date,
#               button_plot,
              error_label,
              fig3
             ])

vbox4 = widgets.VBox([label2,
                      add_equips,
                      selected_equips,
                      label3,
                      selected_start_date,
                      label4,
                      selected_end_date,
                      error_label
                     ])

vbox_fig = widgets.VBox([fig,tb,save_box], align_self='stretch',justify_content='center')
vbox_fig2 = widgets.VBox([fig2,tb2,save_box], align_self='stretch')#,layout=center_layout)
vbox_fig4 = widgets.VBox([fig4,tb4,save_box], align_self='stretch')#,layout=center_layout)
save_box.layout.visibility = 'hidden'

help_button = widgets.Button(
    description = '',
    tooltip='Ajuda',
    icon = 'question-circle',
    button_style='info',
    layout=widgets.Layout(width='35px',align_self='flex-end'),
    disabled=True
)
help1_file = open("help1.txt",'r',encoding='utf-8') 
help_text1 = ''.join(map(str, help1_file.readlines()))
help2_file = open("help2.txt",'r',encoding='utf-8') 
help_text2 = ''.join(map(str, help2_file.readlines()))
help3_file = open("help3.txt",'r',encoding='utf-8') 
help_text3 = ''.join(map(str, help3_file.readlines()))

help_text = widgets.Textarea(
    value=help_text1,
    placeholder='',
    description='',
    disabled=True,
    rows=1,
    layout=widgets.Layout(visibility='hidden',width="auto",height="100px")
)

def on_click_help(b):
    if help_text.layout.visibility=='hidden':
        help_text.layout.visibility='visible'
        app.pane_heights = [0,9,4]
        app2.pane_heights = [0,9,4]
        app4.pane_heights = [0,9,4]
    else:
        help_text.layout.visibility='hidden'
    
help_button.on_click(on_click_help)

app = widgets.AppLayout(header=None,
          left_sidebar=vbox,
          center=vbox_fig,
          right_sidebar=help_button,
          footer=help_text,
          pane_widths=[2, 4, 0],
          pane_heights=[0,9,2],
          layout=widgets.Layout(border= ''))

app2 = widgets.AppLayout(header=None,
          left_sidebar=vbox2,
          center=vbox_fig2,
          right_sidebar=help_button,
          footer=help_text,
          pane_widths=[2, 4, 0],
          pane_heights=[0,9,2],
          layout=widgets.Layout(border= ''))

app4 = widgets.AppLayout(header=None,
          left_sidebar=vbox4,
          center=vbox_fig4,
          right_sidebar=help_button,
          footer=help_text,
          pane_widths=[2, 4, 0],
          pane_heights=[0,9,2],
          layout=widgets.Layout(border= ''))

children = [app,app2,app4]
tab.children = children

header = widgets.HBox()
header.children = [title]

main_app = widgets.VBox([header,load_box,tab])
main_app